# Graded Assignment 3: 9 to 5

Time to show off your SQL skills! For each question, copy the SQL query you used and make note of the answer.

## The Dataset

For this assignment, you will be using the Bureau of Labor Statistics (BLS) Current Employment Survey (CES) results which can be found on [Kaggle](https://www.kaggle.com/datasets/bls/employment).

## Business Issue

You are working for the Bureau of Labor Statistics with the United States government and have been approached by your boss with an important meeting request. You have been asked by your supervisor to meet with Dolly Parton whose nonprofit is looking to shed light on the state of employment in the United States. As part of the 9 to 5 project, their research is focused on production and nonsupervisory employees and how those employees fare compared to all employees in the United States. While the data the BLS collects from the CES is publicly available, Dolly Parton and her colleagues need your assistance navigating the thousands of rows in each table in LaborStatisticsDB.yy 

## About the Dataset

This dataset comes directly from the Bureau of Labor Statistics’ Current Employment Survey (CES). Here are some things you need to know:

1. The industry table contains an NAICS code. This is different from the industry code. NAICS stands for North American Industry Classification System.
1. Series ID is composed of multiple different codes. CES stands for Current Employment Survey, the name of the survey which collected the data. The industry code as specified by the BLS and the data type code as specified in the datatype table.

## Set Up

To connect to the database, use the same connection info used during the SQL lessons. 

For the assignment, we will be using `LaborStatisticsDB`.

## Database Exploration

To start with, let’s get to know the database further.

1. Use this space to make note of each table in the database, the columns within each table, each column’s data type, and how the tables are connected. You can write this down or draw a diagram. Whatever method helps you get an understanding of what is going on with `LaborStatisticsDB`.
   
   To add a photo, diagram or document to your file, drop the file into the folder that holds this notebook.  Use the link button to the right of the  </> symbol in the gray part of this cell, the link is just the name of your file.

In [ ]:
annual_2016:
series_id, year, period, value
Values: 2016 data for different metrics 

january_2017:
series_id, year, period, value
Values: January 2017 data for same metrics

series:
series_id (PK), industry_code, data_type_code, supersector_code, seasonal, series_title
Shows type of data series 

industry:
industry_code, industry_name, naics_code
Industry details and descriptions

datatype:
data_type_code, data_type_text
Types of data 

supersector
supersector_code, supersector_name
industry information

Relationships:
annual_2016.series_id → series.series_id
january_2017.series_id → series.series_id
series.industry_code → industry.industry_code
series.data_type_code → datatype.data_type_code
series.supersector_code → supersector.supersector_code

2. What is the datatype for women employees?

In [ ]:
--10--
USE LaborStatisticsDB
SELECT *
from dbo.series 
WHERE series_title like '%women%'

3. What is the series id for  women employees in the commercial banking industry in the financial activities supersector?

In [ ]:
-- there seems to be two different series id's depending on the employee's seasonal designation
-- for seasonal employees (seasonal=S) the series id is = CES5552211010
-- for unseasonal employees (seasonal=U) the series id is = CEU5552211010

USE LaborStatisticsDB
SELECT *
FROM dbo.series AS s
INNER JOIN dbo.industry AS i
    ON s.industry_code = i.industry_code
INNER JOIN dbo.supersector AS sup
    ON s.supersector_code = sup.supersector_code
WHERE i.industry_name = 'Commercial Banking'
  AND sup.supersector_name = 'Financial Activities'
  AND s.series_title LIKE 'women employees';

## Aggregate Your Friends and Code some SQL

Put together the following:

1. How many employees were reported in 2016 in all industries? Round to the nearest whole number.

In [ ]:
-- 2351408916
SELECT ROUND(SUM(value), 0) AS total_employees_2016
FROM LaborStatisticsDB.dbo.annual_2016

2. How many women employees were reported in 2016 in all industries? Round to the nearest whole number. 

In [ ]:
-- 1125490
SELECT ROUND(SUM(value), 0) AS total_women_employees_2016
FROM LaborStatisticsDB.dbo.annual_2016 as a
INNER JOIN LaborStatisticsDB.dbo.series as s
ON a.series_id = s.series_id
WHERE data_type_code = 10;

3. How many production/nonsupervisory employees were reported in 2016? Round to the nearest whole number. 

In [ ]:
-- 1263650
SELECT ROUND(SUM(value), 0) AS total_production_nonsupervisory_employees_2016
FROM LaborStatisticsDB.dbo.annual_2016 as a
INNER JOIN LaborStatisticsDB.dbo.series as s
ON a.series_id = s.series_id
WHERE data_type_code = 6;

4. In January 2017, what is the average weekly hours worked by production and nonsupervisory employees across all industries?

In [ ]:
-- 36.05862068965521
SELECT AVG(value) as avg_weekly_hours
FROM LaborStatisticsDB.dbo.january_2017 as j
INNER JOIN LaborStatisticsDB.dbo.series as s
ON j.series_id = s.series_id
WHERE data_type_code = '7';

5. What is the total weekly payroll for production and nonsupervisory employees across all industries in January 2017? Round to the nearest penny.

In [ ]:
-- 1838753220
USE LaborStatisticsDB;
SELECT ROUND(SUM(value), 2) AS total_weekly_payroll
FROM dbo.january_2017 AS j
INNER JOIN dbo.series AS s
ON j.series_id = s.series_id 
INNER JOIN dbo.datatype AS dt
ON s.data_type_code = dt.data_type_code
WHERE dt.data_type_code = 82;

6. In January 2017, for which industry was the average weekly hours worked by production and nonsupervisory employees the highest? Which industry was the lowest?

In [ ]:
-- Highest: Motor vehicle power train components
-- Lowest: Fitness and recreational sports centers
USE LaborStatisticsDB;
SELECT TOP 1 i.industry_name AS highest_weekly_hours, j.value, s.series_title
FROM dbo.january_2017 AS j
INNER JOIN dbo.series AS s
ON j.series_id = s.series_id
INNER JOIN dbo.industry AS i
ON s.industry_code = i.industry_code
WHERE s.data_type_code = 7 and j.period = 'm01'
ORDER BY j.value DESC;
-- chance order by to ASC for lowest value. 

7. In January 2021, for which industry was the total weekly payroll for production and nonsupervisory employees the highest? Which industry was the lowest?

In [ ]:
-- Highest: Total private
-- Lowest: Coin-operated laundries and drycleaners
USE LaborStatisticsDB;
SELECT TOP 1 i.industry_name AS highest_payroll_industry, j.value, s.series_title
FROM dbo.january_2017 AS j
INNER JOIN dbo.series AS s
ON j.series_id = s.series_id
INNER JOIN dbo.industry AS i
ON s.industry_code = i.industry_code
WHERE s.data_type_code = 82 and j.period = 'm01'
ORDER BY j.value DESC; 
-- ORDER BY ASC for lowest value

## Join in on the Fun

Time to start joining! You can choose the type of join you use, just make sure to make a  note!

1. Join `annual_2016` with `series` on `series_id`. We only want the data in the `annual_2016` table to be included in the result.

In [ ]:
-- Limiting rows returned from query, uncomment the line below to start on your query!
-- SELECT TOP 50 *


-- Uncomment the line below when you are ready to run the query, leaving it as your last!
-- ORDER BY id

SELECT TOP 50 * 
FROM LaborStatisticsDB.dbo.annual_2016 AS a 
LEFT OUTER JOIN LaborStatisticsDB.dbo.series AS s
ON a.series_id = s.series_id
ORDER BY id

Commands completed successfully.

Total execution time: 00:00:00.019

2. Join `series` and `datatype` on `data_type_code`.

In [ ]:
-- Limiting rows returned from query, uncomment the line below to start on your query!
-- SELECT TOP 50 *


-- Uncomment the line below when you are ready to run the query, leaving it as your last!
-- ORDER BY id

SELECT TOP 50 * 
FROM LaborStatisticsDB.dbo.annual_2016 AS a 
LEFT OUTER JOIN LaborStatisticsDB.dbo.series AS s
ON a.series_id = s.series_id
INNER JOIN LaborStatisticsDB.dbo.datatype as dt
ON s.data_type_code = dt.data_type_code
ORDER BY id

3. Join `series` and `industry` on `industry_code`.

In [ ]:
-- Limiting rows returned from query, uncomment the line below to start on your query!
-- SELECT TOP 50 *


-- Uncomment the line below when you are ready to run the query, leaving it as your last!
-- ORDER BY id

SELECT TOP 50 * 
FROM LaborStatisticsDB.dbo.annual_2016 AS a 
LEFT OUTER JOIN LaborStatisticsDB.dbo.series AS s
ON a.series_id = s.series_id
INNER JOIN LaborStatisticsDB.dbo.datatype AS dt
ON s.data_type_code = dt.data_type_code
INNER JOIN LaborStatisticsDB.dbo.industry AS i
ON s.industry_code = i.industry_code
ORDER BY a.id 

## Subqueries, Unions, Derived Tables, Oh My!

1. Write a query that returns the `series_id`, `industry_code`, `industry_name`, and `value` from the `january_2017` table but only if that value is greater than the average value for `annual_2016` of `data_type_code` 82.

In [ ]:
SELECT j.series_id, s.industry_code, i.industry_name, j.value
FROM LaborStatisticsDB.dbo.january_2017 AS j
INNER JOIN LaborStatisticsDB.dbo.series AS s
ON j.series_id = s.series_id
INNER JOIN LaborStatisticsDB.dbo.industry AS i
ON s.industry_code = i.industry_code
WHERE j.value > (
    SELECT AVG(value)
    FROM LaborStatisticsDB.dbo.annual_2016 AS a
    INNER JOIN LaborStatisticsDB.dbo.series AS s2
    ON a.series_id = s2.series_id
    WHERE s2.data_type_code = 82
);

**Optional Bonus Question:** Write the above query as a common table expression!

In [ ]:
-- Optional CTE below
WITH greater_than_avg2016 AS (
    SELECT AVG(value) AS avg_annual
    FROM  LaborStatisticsDB.dbo.annual_2016 AS a
    INNER JOIN LaborStatisticsDB.dbo.series AS s
    ON a.series_id = s.series_id
    WHERE s.data_type_code = 82
)
SELECT j.series_id, s.industry_code, i.industry_name, j.value
FROM LaborStatisticsDB.dbo.january_2017 AS j
INNER JOIN LaborStatisticsDB.dbo.series AS s
ON j.series_id = s.series_id
INNER JOIN LaborStatisticsDB.dbo.industry AS i
ON s.industry_code = i.industry_code
WHERE j.value > (SELECT avg_annual FROM greater_than_avg2016);

2. Create a `Union` table comparing average weekly earnings of production and nonsupervisory employees between `annual_2016` and `january_2017` using the data type 30.  Round to the nearest penny.  You should have a column for the average earnings and a column for the year, and the period.

In [ ]:
SELECT ROUND(AVG(a.value), 2) AS avg_earnings, a.year, a.period
FROM LaborStatisticsDB.dbo.annual_2016 AS a
INNER JOIN LaborStatisticsDB.dbo.series AS s
ON a.series_id = s.series_id
WHERE s.data_type_code = 30
GROUP BY a.year, a.period

UNION ALL

SELECT ROUND(AVG(j.value), 2) AS avg_earnings, j.year, j.period
FROM LaborStatisticsDB.dbo.january_2017 AS j
INNER JOIN LaborStatisticsDB.dbo.series AS s
ON j.series_id = s.series_id
WHERE s.data_type_code = 30
GROUP BY j.year,j.period;

## Summarize Your Results

With what you know now about the  Bureau of Labor Statistics (BLS) Current Employment Survey (CES) results and working with the Labor Statistics Database, answer the following questions. Note that while this is subjective, you should include relevant data to back up your opinion.

1. During which time period did production and nonsupervisory employees fare better?

2. In which industries did production and nonsupervisory employees fare better?

3. Now that you have explored the datasets, is there any data or information that you wish you had in this analysis?